\title{Digital Latches with myHDL}
\author{Steven K Armour}
\maketitle

# Refs
@book{brown_vranesic_2014, place={New York, NY}, edition={3}, title={Fundamentals of digital logic with Verilog design}, publisher={McGraw-Hill}, author={Brown, Stephen and Vranesic, Zvonko G}, year={2014} },
@book{lameres_2017, title={Introduction to logic circuits & logic design with Verilog}, publisher={springer}, author={LaMeres, Brock J}, year={2017} }

# Acknowledgments

Author of **myHDL** [Jan Decaluwe](http://www.myhdl.org/users/jandecaluwe.html) and the author of the **myHDL Peeker** [XESS Corp.](https://github.com/xesscorp/myhdlpeek)

[**Draw.io**](https://www.draw.io/)

**Xilinx**

# Python Libraries Utilized

In [1]:
import numpy as np
import pandas as pd
from sympy import *
init_printing()

from myhdl import *
from myhdlpeek import *
import random

#python file of convince tools. Should be located with this notebook
from sympy_myhdl_tools import *

# Latches vs Flip-Flops

Latches and Flip-Flops are both metastaple logic circuit tobologies in that once loaded with a state they hold that state information till that state is upset by a new state or a reset command. But the diffrance between the two is that Flip-Flops are clock controlled devices built upon Latches where as Latches are not clock dependent

# SR-Latch

## Symbol and Internals
The Symbol for a SR-Latch and one representation of it's internals is shown below
<img style="float: center;" src="SRLatchSymbolInternal.jpg">

## Definition

## State Diagram

## myHDL SR-Latch Gate and Testing


Need Help Getting this Latch via Combo Cirucits working geting  AlwayCombError in using out signal as argument in out signals next state out

## myHDL SR-Latch Behavioral and Testing

In [2]:
def SRLatch(S_in, rst, Q_out, Qn_out):
    @always_comb
    def logic():
        if S_in and rst==0:
            Q_out.next=1
            Qn_out.next=0
        
        elif S_in==0 and rst:
            Q_out.next=0
            Qn_out.next=1

        elif S_in and rst:
            Q_out.next=0
            Qn_out.next=0

    return logic

In [3]:
S_in, rst, Q_out, Qn_out=[Signal(bool(0)) for _ in range(4)]
Peeker.clear()

Peeker(S_in, 'S_in'); Peeker(rst, 'rst')
Peeker(Q_out, 'Q_out'); Peeker(Qn_out, 'Qn_out')

DUT=SRLatch(S_in=S_in, rst=rst, Q_out=Q_out, Qn_out=Qn_out)
inputs=[S_in, rst]
sim=Simulation(DUT, Combo_TB(inputs), *Peeker.instances()).run()        
Peeker.to_wavedrom(start_time=0, stop_time=2*2**len(inputs), tock=True,
                  title='SRLatch Behavioral simulation',
                  caption=f'after clock cycle {2**len(inputs)-1} ->random input')


<class 'myhdl.StopSimulation'>: No more events


In [4]:
MakeDFfromPeeker(Peeker.to_wavejson(start_time=0, stop_time=2**len(inputs) -1))

,Q_out,Qn_out,S_in,rst
0,0,0,0,0
1,0,1,0,1
2,1,0,1,0
3,0,0,1,1


## myHDL SR-Latch Behavioral HDL Synthesis

In [5]:
toVerilog(SRLatch, S_in, rst, Q_out, Qn_out)
#toVHDL(SRLatch, S_in, rst, Q_out, Qn_out)
_=VerilogTextReader('SRLatch')

***Verilog modual from SRLatch.v***

 // File: SRLatch.v
// Generated by MyHDL 0.9.0
// Date: Thu Oct 26 00:40:43 2017


`timescale 1ns/10ps

module SRLatch (
    S_in,
    rst,
    Q_out,
    Qn_out
);


input S_in;
input rst;
output Q_out;
reg Q_out;
output Qn_out;
reg Qn_out;






always @(rst, S_in) begin: SRLATCH_LOGIC
    if ((S_in && (rst == 0))) begin
        Q_out = 1;
        Qn_out = 0;
    end
    else if (((S_in == 0) && rst)) begin
        Q_out = 0;
        Qn_out = 1;
    end
    else if ((S_in && rst)) begin
        Q_out = 0;
        Qn_out = 0;
    end
end

endmodule



The following shows the **Xilinx**'s _Vivado 2016.1_ RTL generated schematic of our Behaviorla SRLatch from the synthesised verilog code. We can see that the systhizied version is quite apstract from fig lakdfjkaj. 
<img style="float: center;" src="SRLatchBehaviroalRTLSch.PNG">

# Gated SR-Latch

## myHDL SR-Latch Behavioral and Testing

In [6]:
def GSRLatch(S_in, rst, ena, Q_out, Qn_out):
    @always_comb
    def logic():
        if ena:
            if S_in and rst==0:
                Q_out.next=1
                Qn_out.next=0

            elif S_in==0 and rst:
                Q_out.next=0
                Qn_out.next=1

            elif S_in and rst:
                Q_out.next=0
                Qn_out.next=0
        else:
            pass

    return logic

In [7]:
S_in, rst, ena, Q_out, Qn_out=[Signal(bool(0)) for _ in range(5)]
Peeker.clear()

Peeker(S_in, 'S_in'); Peeker(rst, 'rst'); Peeker(ena, 'ena')
Peeker(Q_out, 'Q_out'); Peeker(Qn_out, 'Qn_out')

DUT=GSRLatch(S_in=S_in, rst=rst, ena=ena, Q_out=Q_out, Qn_out=Qn_out)
inputs=[S_in, rst, ena]
sim=Simulation(DUT, Combo_TB(inputs), *Peeker.instances()).run()        
Peeker.to_wavedrom(start_time=0, stop_time=2*2**len(inputs), tock=True,
                  title='GSRLatch Behavioral simulation',
                  caption=f'after clock cycle {2**len(inputs)-1} ->random input')


<class 'myhdl.StopSimulation'>: No more events


In [8]:
MakeDFfromPeeker(Peeker.to_wavejson(start_time=0, stop_time=2**len(inputs) -1))

,Q_out,Qn_out,S_in,ena,rst
0,0,0,0,0,0
1,0,0,0,1,0
2,0,0,0,0,1
3,0,1,0,1,1
4,0,1,1,0,0
5,1,0,1,1,0
6,1,0,1,0,1
7,0,0,1,1,1


## myHDL SR-Latch Behavioral HDL Synthesis

In [9]:
toVerilog(GSRLatch, S_in, rst, ena, Q_out, Qn_out)
#toVHDL(GSRLatch, S_in, rst,ena, Q_out, Qn_out)
_=VerilogTextReader('GSRLatch')

***Verilog modual from GSRLatch.v***

 // File: GSRLatch.v
// Generated by MyHDL 0.9.0
// Date: Thu Oct 26 00:40:44 2017


`timescale 1ns/10ps

module GSRLatch (
    S_in,
    rst,
    ena,
    Q_out,
    Qn_out
);


input S_in;
input rst;
input ena;
output Q_out;
reg Q_out;
output Qn_out;
reg Qn_out;






always @(ena, S_in, rst) begin: GSRLATCH_LOGIC
    if (ena) begin
        if ((S_in && (rst == 0))) begin
            Q_out = 1;
            Qn_out = 0;
        end
        else if (((S_in == 0) && rst)) begin
            Q_out = 0;
            Qn_out = 1;
        end
        else if ((S_in && rst)) begin
            Q_out = 0;
            Qn_out = 0;
        end
    end
    else begin
        // pass
    end
end

endmodule



The following shows the **Xilinx**'s _Vivado 2016.1_ RTL generated schematic of our Behaviorla Gated SRLatch from the synthesised verilog code. We can see that the systhizied version is quite apstract from fig lakdfjkaj. 
<img style="float: center;" src="GSRLatchBehaviroalRTLSch.PNG">

# D-Latch

## myHDL Behavioral D-Latch and Testing

In [10]:
def DLatch(D_in, ena, Q_out, Qn_out):
    #Normal Qn_out is not specifed since a not gate is so easily implimented
    @always_comb
    def logic():
        if ena:
            Q_out.next=D_in
            Qn_out.next=not D_in
    return logic

In [11]:
D_in, ena, Q_out, Qn_out=[Signal(bool(0)) for _ in range(4)]

Peeker.clear()

Peeker(D_in, 'D_in'); Peeker(ena, 'ena')
Peeker(Q_out, 'Q_out'); Peeker(Qn_out, 'Qn_out')

DUT=DLatch(D_in=D_in, ena=ena, Q_out=Q_out, Qn_out=Qn_out)
inputs=[D_in, ena]
sim=Simulation(DUT, Combo_TB(inputs), *Peeker.instances()).run()        
Peeker.to_wavedrom(start_time=0, stop_time=2*2**len(inputs), tock=True,
                  title='DLatch Behavioral simulation',
                  caption=f'after clock cycle {2**len(inputs)-1} ->random input')


<class 'myhdl.StopSimulation'>: No more events


In [12]:
MakeDFfromPeeker(Peeker.to_wavejson(start_time=0, stop_time=2**len(inputs) -1))

,D_in,Q_out,Qn_out,ena
0,0,0,0,0
1,0,0,1,1
2,1,0,1,0
3,1,1,0,1


## myHDL DLatch Behavioral HDL Synthesis

In [13]:
toVerilog(DLatch, D_in, ena, Q_out, Qn_out)
#toVHDL(DLatch,D_in, ena, Q_out, Qn_out)
_=VerilogTextReader('DLatch')

***Verilog modual from DLatch.v***

 // File: DLatch.v
// Generated by MyHDL 0.9.0
// Date: Thu Oct 26 00:40:46 2017


`timescale 1ns/10ps

module DLatch (
    D_in,
    ena,
    Q_out,
    Qn_out
);


input D_in;
input ena;
output Q_out;
reg Q_out;
output Qn_out;
reg Qn_out;






always @(ena, D_in) begin: DLATCH_LOGIC
    if (ena) begin
        Q_out = D_in;
        Qn_out = (!D_in);
    end
end

endmodule



The following shows the **Xilinx**'s _Vivado 2016.1_ RTL generated schematic of our myHDL Dlatch with a exsplisit $\bar{Q}$ verilog code. Note that becouse  $\bar{Q}$ is not normal declared in HDL code Vivado produced two RTL DLatchs and used a NOT Gate to acount for the negated output
<img style="float: center;" src="DLatchBehavioralRTLSch.PNG">

# Examples